---
# Relatório 02 - Análise de Ativos por Setores - Bloomberg Challenge
---

O objetivo desse relatório é identificar ativos a serem analisados e investidos ao longo do Bloomberg Challenge de Outubro/2024, considerando os setores escolhidos pela análise setorial (Relatório 01)

## 1. Importação de bibliotecas e configuração

### 1.1. Importação de Bibliotecas e Configurações de Data

In [1]:
# Importação de dados
import yfinance as yf 

# Análise de dados
import pandas as pd 

# Visualização
import plotly.graph_objects as go 
import plotly.express as px


# Outros
import os 
from datetime import datetime as dt 

data_inicial = dt(2020, 1, 1)
data_final = dt.today()

### 1.2. Importação de lista de ativos

In [2]:
dir = 'data/ativos_selecionados'

ativos_selecionados = pd.DataFrame()
for arq in os.listdir(dir):
    if arq.endswith('.csv'):
        temp = pd.read_csv(dir + '/' + arq)
        ativos_selecionados = pd.concat([ativos_selecionados, temp])

In [3]:
ativos_selecionados.shape

(354, 12)

In [4]:
ativos_selecionados.head()

,Ticker,Nome,Ponderação,Ações,Preço,MainTicker,YFTicker,SetorEconomico,Industria,NomeCompletoParaAuditoria,Pais,Moeda
0,PROT NO Equity,Protector Forsikring ASA,0.001278,50.451,237.50,PROT,PROT,Healthcare,Biotechnology,PROTEONOMIX INC,United States,USD
1,ALAB UW Equity,Astera Labs Inc,0.001276,21.729,52.10,ALAB,ALAB,Technology,Semiconductors,"Astera Labs, Inc.",United States,USD
2,MXL UW Equity,MaxLinear Inc,0.001275,76.115,14.86,MXL,MXL,Technology,Semiconductors,"MaxLinear, Inc",United States,USD
3,HRMY UQ Equity,Harmony Biosciences Holdings Inc,0.001256,29.877,37.29,HRMY,HRMY,Healthcare,Biotechnology,"Harmony Biosciences Holdings, I",United States,USD
4,TVTX UQ Equity,Travere Therapeutics Inc,0.001227,74.851,14.54,TVTX,TVTX,Healthcare,Biotechnology,"Travere Therapeutics, Inc.",United States,USD


#### 1.2.1. Enriquecimento de dados de bolsa

In [5]:
def add_market_information(company_name):
    """ Função para adicionar o país em que o ativo é operado e a moeda utilizada"""
    try:
        ticker = yf.Ticker(company_name)
    except:
        print(company_name)
        raise
    market = ticker.info.get('exchange', 'N/A')

    return market

In [6]:
ativos_selecionados['Bolsa'] = ativos_selecionados['YFTicker'].apply(lambda x: pd.Series(add_market_information(x)))

### 1.3. Importação de Dados OHLCV

In [7]:
dados_ohlcv = {}

for ativo in ativos_selecionados['YFTicker'].unique():
    temp = yf.download(ativo, start=data_inicial.strftime('%Y-%m-%d'), end=data_final.strftime('%Y-%m-%d'), progress=False)
    if not temp.empty:
        dados_ohlcv[ativo] = temp 

len(dados_ohlcv)

295

Como visto acima, alguns ativos não foram importados e, por isso, não estarão sujeitos a essa análise.

## 2. Análise Exploratória dos Ativos

In [8]:
dados_ohlcv.keys()

dict_keys(['PROT', 'ALAB', 'MXL', 'HRMY', 'TVTX', 'KPRX', 'ACT', 'AVIR', 'LBPH', 'INVA', 'YMAB', 'SGH', 'PRAX', 'WOLF', 'COGT', 'TGI', 'NMRA', 'ARQT', 'LMND', 'RLAY', 'SPR', 'AMSF', 'ZYME', 'SPRY', 'MRVI', 'HCI', 'QLGN', 'DAWN', 'IRON', 'CBUS', 'CDRE', 'ACET', 'AOSL', 'ELVN', 'DCO', 'VIR', 'PLMR', 'RCUS', 'ENTA', 'CGEM', 'PRA', 'ARVN', 'STC', 'RGR', 'IMNM', 'CKPT', 'SILO', 'VVX', 'DNTH', 'MDXG', 'ERAS', 'PLRX', 'BA', 'ANAB', 'OLMA', 'AURA', 'MRNA', 'GE', 'ETNB', 'SRRK', 'SWBI', 'ATRO', 'ORIC', 'FDMT', 'SMTC', 'IBRX', 'ACHR', 'LQDA', 'ADPT', 'ATXS', 'ARCT', 'TNGX', 'SANA', 'RGNX', 'AMBC', 'CVM', 'HXL', 'TYRA', 'HUMA', 'THG', 'NUVB', 'ACAD', 'SYRS', 'GLSI', 'SAGE', 'ZVRA', 'ALLO', 'SLRN', 'TPST', 'PRTC', 'JSPR', 'HGTY', 'DNA', 'CMRX', 'PTGX', 'NVTS', 'LXRX', 'FHTX', 'ROOT', 'INAB', 'IGMS', 'MBRX', 'GNPX', 'PRME', 'GYRE', 'AIP', 'SGMO', 'AEON', 'PWI.F', 'TXT', 'BMEA', 'EVEX', 'ORI', 'FGEN', 'TSHA', 'SAVA', 'TTNP', 'NVDA', 'AVGO', 'AMD', 'QCOM', 'TXN', 'RTX', 'PGR', 'LMT', 'VRTX', 'ADI', '

### 2.1. Obtenção dos Volumes Médios Diários

Um dos critérios possíveis é a escolha de ativos dos setores selecionados que possua volumes suficientes para a modelagem. 
Quanto mais operações, maior a chance de o mercado não ser afetado por um player único.

In [9]:
volumes_medios = {}
for ativo in dados_ohlcv.keys():
    volumes_medios[ativo] = dados_ohlcv[ativo]['Volume'].mean()

volumes_medios = pd.DataFrame(volumes_medios, index = ['Média Volume']).T.sort_values(by='Média Volume', ascending=False)
volumes_medios.reset_index(inplace=True)
volumes_medios.columns = ['YFTicker', 'Média Volume']
volumes_medios.head(20)

,YFTicker,Média Volume
0,NVDA,4.577784e+08
1,AMD,6.507724e+07
2,INTC,3.916639e+07
3,AVGO,2.465012e+07
4,MU,1.973214e+07
5,BA,1.287788e+07
6,GE,1.169423e+07
7,MRVL,1.047801e+07
8,MRNA,9.476764e+06
9,QCOM,9.137129e+06


#### 2.1.1. Obtenção de volumes de ativos com indicação de setor

In [10]:
dados_enriquecimento = ativos_selecionados[['YFTicker', 'Industria', 'SetorEconomico', 'NomeCompletoParaAuditoria', 'Bolsa']].drop_duplicates(subset='YFTicker')
vol_med_ref = volumes_medios.merge(dados_enriquecimento, on='YFTicker', how = 'left', validate='one_to_one')
vol_med_ref.head()

,YFTicker,Média Volume,Industria,SetorEconomico,NomeCompletoParaAuditoria,Bolsa
0,NVDA,4.577784e+08,Semiconductors,Technology,NVIDIA Corporation,NMS
1,AMD,6.507724e+07,Semiconductors,Technology,"Advanced Micro Devices, Inc.",NMS
2,INTC,3.916639e+07,Semiconductors,Technology,Intel Corporation,NMS
3,AVGO,2.465012e+07,Semiconductors,Technology,Broadcom Inc.,NMS
4,MU,1.973214e+07,Semiconductors,Technology,"Micron Technology, Inc.",NMS


In [11]:
# Obtendo mediana por setor 
ativos_permitidos = {}

for industria in vol_med_ref['Industria']:
    mediana = vol_med_ref[vol_med_ref['Industria']==industria]['Média Volume'].median()
    ativos_permitidos[industria] = list(vol_med_ref[(vol_med_ref['Industria']==industria) & (vol_med_ref['Média Volume']>= mediana)]['YFTicker'].values)


### 2.2. Distribuição dos Volumes

In [12]:
fig = px.box(vol_med_ref, x="Industria", y="Média Volume", title="Box Plot do Volume Médio por Industria", template="plotly_dark")
fig.show()

Percebe-se que NVIDIA tem sido um outlier em questão de volume

In [13]:
fig = px.box(vol_med_ref, x="Bolsa", y="Média Volume", title="Box Plot do Volume Médio por Bolsa", template="plotly_dark")
fig.show()

O que afeta, inclusive, o comportamento da média de volume da bolsa respectiva (NMS).

## 3. Escolha dos ativos

In [14]:
betas = pd.read_excel(r'data\ativos_selecionados\Ativos x Beta(1).xlsx', sheet_name='WLS COMPLETO')
betas.head()

,Ticker,MainTicker,Bolsa,Nome,Ponderação,Ações,Preço,Beta Corrigido,Beta,YFTicker,SetorEconomico,Industria,NomeCompletoParaAuditoria
0,ISMT IS Equity,ISMT,IS,ISMT Ltd,--,56.490,--,1.260681,1.26066,0,0,0,0
1,TCNSBR IS Equity,TCNSBR,IS,TCNS Clothing Co Ltd,--,12.980,--,0.685257,0.685245,TCNSBRANDS.BO,Consumer Cyclical,Apparel Manufacturing,TCNS Clothing Co. Limited
2,AAPL UW Equity,AAPL,UW,Apple Inc,3.748542,14793.489,226.21,1.206749,1.206733,AAPL,Technology,Consumer Electronics,Apple Inc.
3,MSFT UW Equity,MSFT,UW,Microsoft Corp,3.498732,7424.523,420.69,1.13058,1.130565,MSFT,Technology,Software—Infrastructure,Microsoft Corporation
4,NVDA UW Equity,NVDA,UW,NVIDIA Corp,3.096586,23627.475,117,2.221154,2.221128,NVDA,Technology,Semiconductors,NVIDIA Corporation


In [15]:
ativos_selecionados = betas[betas['YFTicker'].isin(ativos_selecionados['YFTicker'])].sort_values(by='Beta', ascending=False)
ativos_selecionados.head()

,Ticker,MainTicker,Bolsa,Nome,Ponderação,Ações,Preço,Beta Corrigido,Beta,YFTicker,SetorEconomico,Industria,NomeCompletoParaAuditoria
2871,TGTX UR Equity,TGTX,UR,TG Therapeutics Inc,0.003573,139.734,22.83,3.267937,3.267901,TGTX,Healthcare,Biotechnology,"TG Therapeutics, Inc."
4651,RXRX UW Equity,RXRX,UW,Recursion Pharmaceuticals Inc,0.001555,224.990,6.17,3.239381,3.239343,RXRX,Healthcare,Biotechnology,"Recursion Pharmaceuticals, Inc."
3006,SITM UQ Equity,SITM,UQ,SiTime Corp,0.003341,17.728,168.25,2.86347,2.863438,SITM,Technology,Semiconductors,SiTime Corporation
6039,LMND UN Equity,LMND,UN,Lemonade Inc,0.000947,52.360,16.15,2.835503,2.835469,LMND,Financial Services,Insurance—Property & Casualty,"Lemonade, Inc."
9243,IGMS UW Equity,IGMS,UW,IGM Biosciences Inc,0.000229,14.116,14.5,2.821168,2.821137,IGMS,Healthcare,Biotechnology,"IGM Biosciences, Inc."


### 3.1. Separação de ativos por betas

In [16]:
beta_alto = ativos_selecionados[ativos_selecionados['Beta']>1].merge(volumes_medios, on='YFTicker', how='left')
beta_baixo = ativos_selecionados[(ativos_selecionados['Beta']<1)&(ativos_selecionados['Beta']>0.5)].merge(volumes_medios, on='YFTicker', how='left')

In [17]:
ativos_final = pd.DataFrame()

for df in [beta_alto, beta_baixo]:
    for industria in df['Industria']:
        temp = df[(df['Industria']==industria) & (df['YFTicker'].isin(ativos_permitidos[industria]))].head(12)
        ativos_final = pd.concat([ativos_final, temp])

ativos_final.drop_duplicates(inplace=True)

In [28]:
ativos_final.reset_index(drop=True)['Industria'].value_counts()

Industria
Biotechnology                    24
Aerospace & Defense              19
Semiconductors                   13
Insurance—Property & Casualty     7
Insurance—Specialty               5
Insurance Brokers                 1
Name: count, dtype: int64

Filtrando dados pelos ativos selecionados

In [31]:
DADOS_INPUT = {ativo:df for ativo, df in dados_ohlcv.items() if ativo in (ativos_final['YFTicker'].values)}
len(DADOS_INPUT.keys())

61

In [ ]:
DADOS_INPUT